<a href="https://colab.research.google.com/github/bhavanapagadala/hackton/blob/main/satellite_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install folium geopy

import folium
from folium.plugins import Search
from IPython.display import display
from geopy.geocoders import Nominatim

# Sample disaster + safe zone data with rescue numbers
disaster_data = [
    {"city": "Delhi", "lat": 28.6139, "lon": 77.2090, "severity": "High", "safe_zone": "Indira Stadium", "safe_lat": 28.6330, "safe_lon": 77.2185, "rescue_number": "011-100"},
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "severity": "Critical", "safe_zone": "Wankhede Grounds", "safe_lat": 19.0000, "safe_lon": 72.8200, "rescue_number": "022-100"},
    {"city": "Chennai", "lat": 13.0827, "lon": 80.2707, "severity": "Moderate", "safe_zone": "Marina Shelter", "safe_lat": 13.0400, "safe_lon": 80.2800, "rescue_number": "044-100"},
    {"city": "Kolkata", "lat": 22.5726, "lon": 88.3639, "severity": "Severe", "safe_zone": "Netaji Stadium", "safe_lat": 22.5600, "safe_lon": 88.3700, "rescue_number": "033-100"},
    {"city": "Coimbatore", "lat": 11.0168, "lon": 76.9558, "severity": "Moderate", "safe_zone": "PSG Auditorium", "safe_lat": 11.018, "safe_lon": 76.9600, "rescue_number": "0422-100"},
    {"city": "Kottur", "lat": 10.9331, "lon": 76.9565, "severity": "Severe", "safe_zone": "Kottur School Ground", "safe_lat": 10.936, "safe_lon": 76.9588, "rescue_number": "0462-100"}
]

severity_colors = {
    "Critical": "darkred",
    "Severe": "orange",
    "High": "red",
    "Moderate": "blue",
    "Low": "green"
}

# Initialize map
disaster_map = folium.Map(location=[22.5, 78.5], zoom_start=5)

# Group for searchable disaster markers
feature_group = folium.FeatureGroup(name="Disasters")
disaster_map.add_child(feature_group)

# Add disaster markers, safe zones, and routes
for entry in disaster_data:
    color = severity_colors.get(entry["severity"], "gray")

    folium.Marker(
        location=[entry["lat"], entry["lon"]],
        tooltip=entry["city"],
        popup=folium.Popup(f"""
            <b>City:</b> {entry['city']}<br>
            <b>Severity:</b> <span style='color:{color}'>{entry['severity']}</span><br>
            <b>Safe Zone:</b> {entry['safe_zone']}<br>
            <b>Rescue Number:</b> <a href='tel:{entry['rescue_number']}'>{entry['rescue_number']}</a>
        """, max_width=300),
        icon=folium.Icon(color=color)
    ).add_to(feature_group)

    folium.Marker(
        location=[entry["safe_lat"], entry["safe_lon"]],
        tooltip=f"✅ {entry['safe_zone']}",
        popup=f"Safe Zone for {entry['city']}",
        icon=folium.Icon(color="green", icon="ok-sign")
    ).add_to(disaster_map)

    folium.PolyLine(
        locations=[[entry["lat"], entry["lon"]], [entry["safe_lat"], entry["safe_lon"]]],
        color="blue", weight=2, tooltip="Evacuation Route"
    ).add_to(disaster_map)

# Add search functionality for disaster markers only
search = Search(
    layer=feature_group,
    search_label="tooltip",
    placeholder="Search disaster location...",
    collapsed=False
)
disaster_map.add_child(search)

# Layer toggle
folium.LayerControl().add_to(disaster_map)

# Display in Colab
display(disaster_map)


In [5]:
!pip install folium geopy

import folium
from folium.plugins import Search
from IPython.display import display
from geopy.distance import geodesic

# Sample data
disaster_data = [
    {"city": "Delhi", "lat": 28.6139, "lon": 77.2090, "severity": "High", "safe_zone": "Indira Stadium", "safe_lat": 28.6330, "safe_lon": 77.2185, "rescue_number": "011-100"},
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "severity": "Critical", "safe_zone": "Wankhede Grounds", "safe_lat": 19.0000, "safe_lon": 72.8200, "rescue_number": "022-100"},
    {"city": "Chennai", "lat": 13.0827, "lon": 80.2707, "severity": "Moderate", "safe_zone": "Marina Shelter", "safe_lat": 13.0400, "safe_lon": 80.2800, "rescue_number": "044-100"},
    {"city": "Kolkata", "lat": 22.5726, "lon": 88.3639, "severity": "Severe", "safe_zone": "Netaji Stadium", "safe_lat": 22.5600, "safe_lon": 88.3700, "rescue_number": "033-100"},
    {"city": "Coimbatore", "lat": 11.0168, "lon": 76.9558, "severity": "Moderate", "safe_zone": "PSG Auditorium", "safe_lat": 11.018, "safe_lon": 76.9600, "rescue_number": "0422-100"},
    {"city": "Kottur", "lat": 10.9331, "lon": 76.9565, "severity": "Severe", "safe_zone": "Kottur School Ground", "safe_lat": 10.936, "safe_lon": 76.9588, "rescue_number": "0462-100"}
]

severity_colors = {
    "Critical": "darkred",
    "Severe": "orange",
    "High": "red",
    "Moderate": "blue",
    "Low": "green"
}

severity_radius = {
    "Critical": 10000,
    "Severe": 7000,
    "High": 5000,
    "Moderate": 3000,
    "Low": 2000
}

# Initialize map
disaster_map = folium.Map(location=[22.5, 78.5], zoom_start=5)

# Add satellite layer
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri Satellite',
    name='Satellite',
    overlay=False,
    control=True
).add_to(disaster_map)

# Add weather temperature overlay (OpenWeatherMap demo tile)
folium.TileLayer(
    tiles="https://tile.openweathermap.org/map/temp_new/{z}/{x}/{y}.png?appid=439d4b804bc8187953eb36d2a8c26a02",
    attr="OpenWeatherMap",
    name="Temperature",
    overlay=True,
    control=True
).add_to(disaster_map)

# Feature group
feature_group = folium.FeatureGroup(name="Disasters").add_to(disaster_map)

for entry in disaster_data:
    color = severity_colors.get(entry["severity"], "gray")
    radius = severity_radius.get(entry["severity"], 2000)

    # Distance calculation
    dist_km = round(geodesic((entry["lat"], entry["lon"]), (entry["safe_lat"], entry["safe_lon"])).km, 2)

    # Disaster Marker
    folium.Marker(
        location=[entry["lat"], entry["lon"]],
        tooltip=entry["city"],
        popup=folium.Popup(f"""
            <b>City:</b> {entry['city']}<br>
            <b>Severity:</b> <span style='color:{color}'>{entry['severity']}</span><br>
            <b>Safe Zone:</b> {entry['safe_zone']}<br>
            <b>Rescue Number:</b> <a href='tel:{entry['rescue_number']}'>{entry['rescue_number']}</a><br>
            <b>Distance to Safe Zone:</b> {dist_km} km
        """, max_width=300),
        icon=folium.Icon(color=color)
    ).add_to(feature_group)

    # Safe Zone Marker
    folium.Marker(
        location=[entry["safe_lat"], entry["safe_lon"]],
        tooltip=f"✅ {entry['safe_zone']}",
        popup=f"Safe Zone for {entry['city']}",
        icon=folium.Icon(color="green", icon="ok-sign")
    ).add_to(disaster_map)

    # Evacuation Route
    folium.PolyLine(
        locations=[[entry["lat"], entry["lon"]], [entry["safe_lat"], entry["safe_lon"]]],
        color="blue", weight=2, tooltip=f"{dist_km} km route"
    ).add_to(disaster_map)

    # Circle around disaster zone
    folium.Circle(
        location=[entry["lat"], entry["lon"]],
        radius=radius,
        color=color,
        fill=True,
        fill_opacity=0.2,
        popup=f"{entry['severity']} zone"
    ).add_to(disaster_map)

# Search + Layer control
Search(layer=feature_group, search_label="tooltip", placeholder="Search city...", collapsed=False).add_to(disaster_map)
folium.LayerControl().add_to(disaster_map)

# Add Legend
legend_html = '''
<div style="position: fixed; bottom: 40px; left: 40px; z-index:9999; background-color: white; padding: 10px; border:2px solid grey;">
    <h4>Severity Legend</h4>
    <i style="background:darkred;width:15px;height:15px;display:inline-block;"></i> Critical<br>
    <i style="background:orange;width:15px;height:15px;display:inline-block;"></i> Severe<br>
    <i style="background:red;width:15px;height:15px;display:inline-block;"></i> High<br>
    <i style="background:blue;width:15px;height:15px;display:inline-block;"></i> Moderate<br>
</div>
'''
disaster_map.get_root().html.add_child(folium.Element(legend_html))

# Display and save
disaster_map.save("disaster_map.html")
disaster_map
